# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [108]:
import gensim
from tqdm import tqdm
import numba
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [109]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2021-04-17 01:22:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.4’

shakespeare_plays.t 100%[===================>]   4.53M  --.-KB/s    in 0.1s    

2021-04-17 01:22:08 (46.8 MB/s) - ‘shakespeare_plays.txt.4’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [110]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [111]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.80985426902771),
 ('duke', 0.7773141860961914),
 ('york', 0.7664082646369934),
 ('prince', 0.7646195888519287),
 ('gloucester', 0.742842435836792),
 ('cardinal', 0.7427535057067871),
 ('warwick', 0.7411811351776123),
 ('clarence', 0.7070386409759521),
 ('widow', 0.7051149606704712),
 ('suffolk', 0.693950891494751)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [112]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9518844485282898),
 ('iago', 0.9488846659660339),
 ('emilia', 0.9115982055664062),
 ('cassio', 0.9101124405860901),
 ('ham', 0.894819974899292),
 ('cleopatra', 0.8941190838813782),
 ('fal', 0.8876879811286926),
 ('ha', 0.8874792456626892),
 ('aside', 0.8828766345977783),
 ('troilus', 0.882326602935791)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [113]:
model.wv.similarity('othello', 'desdemona')

0.9518845

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [114]:
## TODO: Implement cosim
@numba.jit(target = 'cpu', nopython = True)
def cosim(v1, v2):
  mag = np.sqrt(np.dot(v1,v1)) * np.sqrt(np.dot(v2,v2))
  similarity = np.dot(v1,v2) / mag
  ## return cosine similarity between v1 and v2
  return similarity

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])


/usr/local/lib/python3.7/dist-packages/numba/core/decorators.py:150: NumbaDeprecationWarning: The 'target' keyword argument is deprecated.
  warnings.warn("The 'target' keyword argument is deprecated.", NumbaDeprecationWarning)


0.9518845081329346

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [115]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2021-04-17 01:22:23--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.5’

30k.en.vec.5        100%[===================>]  64.54M   252MB/s    in 0.3s    

2021-04-17 01:22:24 (252 MB/s) - ‘30k.en.vec.5’ saved [67681172/67681172]

--2021-04-17 01:22:24--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.5’

30k.fr.vec.5        100%[===================>]  64.66M   294MB/s    in 0.2s    

2021-04-17 01:22:24 (294 MB/s) - ‘30k.fr.vec.5’ saved [67802327/67802327]

--2021-04-17 01:22:24--  http://www.cc

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [116]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec') ## Spanish
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [117]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  K=1
  similarities = {k : cosim(vec, v) for k, v in vecDict.items()}
  return (sorted(similarities.items(), key = 
             lambda kv:(kv[1], kv[0]))[-1])
  
## some example searches
print("--------------------For English and French------------------------")
print([mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']])
print("--------------------For English and Spanish------------------------")
print([mostSimilar(envec[e], esvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']])

--------------------For English and French------------------------
[('informatique', 0.5023827767603765), ('allemagne', 0.593718413875964), ('matrice', 0.5088361302065517), ('physique', 0.4555543434796394), ('fermentation', 0.3504105196166514)]
--------------------For English and Spanish------------------------
[('computador', 0.5013697495254124), ('alemania', 0.6352798713596078), ('matriz', 0.4784864671614966), ('física', 0.4784845095690361), ('levadura', 0.5114917452709493)]


Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [118]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2021-04-17 01:22:36--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.4’

links.tab.4         100%[===================>]   1.34M  --.-KB/s    in 0.04s   

2021-04-17 01:22:37 (34.5 MB/s) - ‘links.tab.4’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [119]:
links[302]

['academy', 'fr', 'académie']


**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [136]:
count_total = 0
count_baseline = 0
wikiEnToFr = dict()
english = []
french = []
for i in range(len(links)):
  if links[i][1] == 'fr':
    wikiEnToFr[links[i][0]] = links[i][2]
    english.append(links[i][0])
    french.append(links[i][2])

for i in wikiEnToFr:
  if i == wikiEnToFr[i]:
    count_baseline = count_baseline + 1
  count_total += 1

baseline_accuracy = count_baseline/ count_total
print("BASELINE ACCURACY for English and French : {}".format(baseline_accuracy))


BASELINE ACCURACY for English and French : 0.6742324450298915


In [143]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
def base_dict(envec, lang2vec, english):
  eval_list = []
  for i in english:
    similar = ''
    if i in envec:
      similar = mostSimilar(envec[i],lang2vec)[0]
      eval_list.append(similar)
    else:
      eval_list.append(None)
  return eval_list

french_list = base_dict(envec, frvec, english)

acc_count = 0
total_count = 0
for i in range(len(french_list)):
  if french_list[i] == french[i]:
    acc_count += 1
  total_count += 1
  
print("ACCURACY FOR ENGLISH AND FRENCH :{}", format(acc_count/ total_count))


ACCURACY FOR ENGLISH AND FRENCH :{} 0.5359205593271862


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [145]:
## TODO: Compute English-X Wikipedia BASELINE accuracy.
count_total = 0
count_baseline = 0
wikiEnToEs = dict()
english = []
spanish = []
for i in range(len(links)):
  if links[i][1] == 'es':
    wikiEnToEs[links[i][0]] = links[i][2]
    english.append(links[i][0])
    spanish.append(links[i][2])

for i in wikiEnToEs:
  if i == wikiEnToEs[i]:
    count_baseline = count_baseline + 1
  count_total += 1

baseline_accuracy = count_baseline/ count_total
print("BASELINE ACCURACY for English and Spanish : {}".format(baseline_accuracy))

BASELINE ACCURACY for English and Spanish : 0.5173403193612774


In [147]:
## TODO: Compute English-X Wikipedia retrieval accuracy.

spanish_list = base_dict(envec, esvec,  english)

acc_count = 0
total_count = 0
for i in range(len(spanish_list)):
  if spanish_list[i] == spanish[i]:
    acc_count += 1
  total_count += 1
  
print("ACCURACY FOR ENGLISH AND SPANISH :{}", format(acc_count/ total_count))

ACCURACY FOR ENGLISH AND SPANISH :{} 0.5432884231536926


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".